# 第10章 — ニューラルネットワーク

- パーセプトロンは重み付き和に活性化をかけ、層を重ねて関数を近似する。
- 学習では損失の勾配降下で重みを更新する。
- 非線形活性化（ReLUやシグモイド）が曲線的な決定境界を表現する。
- データの正規化と小さめの学習率が安定性を高める。

試してみよう: 活性化をReLUに変えたり、OR/XORを学習して（XORは中間層が必要）挙動を試す。

In [ ]:
# パーセプトロンの定義とシグモイド活性化
import math
import random

def sigmoid(x):
    return 1 / (1 + math.exp(-x))


class Perceptron:
    def __init__(self, w, b=0.0, lr=0.1):
        self.w = w
        self.b = b
        self.lr = lr

    def predict(self, x1, x2):
        z = self.w[0] * x1 + self.w[1] * x2 + self.b
        return sigmoid(z)

    def train(self, data, epochs=20):
        for _ in range(epochs):
            for x1, x2, label in data:
                pred = self.predict(x1, x2)
                error = label - pred
                self.w[0] += self.lr * error * x1
                self.w[1] += self.lr * error * x2
                self.b += self.lr * error


In [ ]:
# ANDゲートのデータで学習と推論
samples = [
    (0, 0, 0),
    (0, 1, 0),
    (1, 0, 0),
    (1, 1, 1),
]

p = Perceptron([0.1, 0.1], b=-0.1, lr=0.5)
p.train(samples, epochs=30)

for x1, x2, _ in samples:
    print((x1, x2), "->", round(p.predict(x1, x2), 3))


### 追加例: 2層パーセプトロンでXOR

In [ ]:
# 2層パーセプトロン(MLP)の定義
def dsigmoid(y):
    return y * (1 - y)


class MLP:
    def __init__(self):
        random.seed(0)
        self.w1 = [[random.uniform(-1, 1) for _ in range(2)] for _ in range(2)]
        self.b1 = [0.0, 0.0]
        self.w2 = [random.uniform(-1, 1) for _ in range(2)]
        self.b2 = 0.0
        self.lr = 0.5

    def forward(self, x1, x2):
        h1 = sigmoid(self.w1[0][0] * x1 + self.w1[0][1] * x2 + self.b1[0])
        h2 = sigmoid(self.w1[1][0] * x1 + self.w1[1][1] * x2 + self.b1[1])
        out = sigmoid(self.w2[0] * h1 + self.w2[1] * h2 + self.b2)
        return (h1, h2), out

    def train(self, data, epochs=5000):
        for _ in range(epochs):
            for x1, x2, label in data:
                (h1, h2), out = self.forward(x1, x2)
                error = label - out
                d_out = error * dsigmoid(out)
                d_h1 = d_out * self.w2[0] * dsigmoid(h1)
                d_h2 = d_out * self.w2[1] * dsigmoid(h2)
                self.w2[0] += self.lr * d_out * h1
                self.w2[1] += self.lr * d_out * h2
                self.b2 += self.lr * d_out
                self.w1[0][0] += self.lr * d_h1 * x1
                self.w1[0][1] += self.lr * d_h1 * x2
                self.w1[1][0] += self.lr * d_h2 * x1
                self.w1[1][1] += self.lr * d_h2 * x2
                self.b1[0] += self.lr * d_h1
                self.b1[1] += self.lr * d_h2


In [ ]:
# XORデータをMLPに学習させて出力を確認
mlp = MLP()
data = [(0, 0, 0), (0, 1, 1), (1, 0, 1), (1, 1, 0)]
mlp.train(data, epochs=3000)

for x1, x2, label in data:
    (_, _), out = mlp.forward(x1, x2)
    print((x1, x2), "->", round(out, 3), "label", label)
